# Конвертирование Numpy Array в TF Records
Сохраните набор данных белка CullPDB в виде записей TF, чтобы упростить их импорт в модели.
Этот notepad представляет собой пошаговое руководство по файлу *make_tfrecords.py*.

In [1]:
import numpy as np
import tensorflow as tf

# определить tf.train.Features 
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [2]:
# Загрузить данные; Я сохранил данные в своем каталоге HOME / data / cpdb
from pathlib import Path
HOME = str(Path.home())

# Измените данные так, чтобы мы получили (num_samples x max_seq_length x num_features)
data = np.load(HOME+"/data/cpdb/cpdb+profile_6133_filtered.npy.gz").reshape(-1, 700, 57)
num_samples = data.shape[0]

# Показать второй остаток первого образца
data[0, 1, :]

array([0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 1.        , 1.        ,
       0.05732417, 0.00669285, 0.9976241 , 0.96832377, 0.00682713,
       0.14185107, 0.09885607, 0.00919371, 0.80059224, 0.00784502,
       0.01763634, 0.55971366, 0.0541667 , 0.4378235 , 0.12786157,
       0.16798161, 0.14931294, 0.01224884, 0.00532502, 0.26894143,
       0.01551975, 0.        ])

In [3]:
# Нас интересуют только особенности матрицы аминокислот и профиля PSI-BLAST
seqs = np.concatenate([data[:, :, 0:22].copy(), data[:, :, 35:56].copy()], axis=2).reshape(num_samples, -1)

# Есть 43 функции на остаток
num_features = 43

seqs[0, 43:86]

array([0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.05732417, 0.00669285, 0.9976241 ,
       0.96832377, 0.00682713, 0.14185107, 0.09885607, 0.00919371,
       0.80059224, 0.00784502, 0.01763634, 0.55971366, 0.0541667 ,
       0.4378235 , 0.12786157, 0.16798161, 0.14931294, 0.01224884,
       0.00532502, 0.26894143, 0.01551975])

In [4]:
# Измените метки, но не выровняйте, чтобы мы могли посчитать NoSeqs позже
labels = data[:, :, 22:31].copy().reshape(num_samples, 700, -1)
labels[0, 1, :]

array([1., 0., 0., 0., 0., 0., 0., 0., 0.])

## Сохранить как запись TF
Мы хотим сохранить каждый семпл как запись TF. Для экономии места и сокращения времени обучения используется
заполнение из исходного набора данных (каждый белок был дополнен до 700 токенами NoSeq).

In [ ]:
# Подсчитайте количество токенов non-NoSeq, чтобы найти длину белка
noseq = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 1.]])

# Найти все позиции в векторе, которые являются меткой «NoSeq»
noseqs = np.all(np.equal(labels[0, :, :], noseq), axis=1)

# Теперь отмените логические значения и посчитайте количество истин
seq_length = np.logical_not(noseqs).sum()
print(seq_length)

noseqs

In [ ]:
# Теперь, когда мы знаем длину, мы можем сгладить метки
labels = labels.reshape(num_samples, -1)


num_labels = 9

# Преобразовать в tf.train.Example, который в конечном итоге будет записан в файл
example = tf.train.Example(features=tf.train.Features(feature={
    'seq_len': _int64_feature(seq_length),
    'seq_data': _floats_feature(seqs[0, 0:num_features*seq_length]),
    'label_data': _floats_feature(labels[0, 0:num_labels*seq_length])
    }
))
example